# Paso 1: Importar Librerías de Python


In [1]:
#pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163480 sha256=9013990b072df26ad90bcafe70edd0abfbbb66968656e15d01043736856ca233
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
# Análisis de datos y manipulación
import pandas as pd
import numpy as np

# Visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns

# Utilidades de fechas
from datetime import datetime

# Integración con Google Colab
from google.colab import drive

# Machine Learning: Preprocesamiento y división de datos
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Machine Learning: Métricas
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity

# Machine Learning: Modelado y validación
from surprise import SVD, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split, GridSearchCV

# Deep Learning: Construcción de modelos y capas
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout, Dot

# Deep Learning: Optimización y regularización
from keras.optimizers import Adam
from keras.regularizers import l2

# Deep Learning: Callbacks
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


# Paso 2: Lectura del Dataset

In [4]:
# Suponiendo que tu DataFrame se llama df
df = pd.read_csv('/content/drive/MyDrive/EspecializacionA&DS/Monografia/2doSemestre/DataFinal_Amazon.csv')  # Descomenta y proporciona la ruta si estás cargando un archivo CSV

<ipython-input-4-1f8e3361d82a>:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/EspecializacionA&DS/Monografia/2doSemestre/DataFinal_Amazon.csv')  # Descomenta y proporciona la ruta si estás cargando un archivo CSV


In [5]:
# Eliminemos posibles duplicados:
df = df.drop_duplicates()

In [6]:
df.shape

(4641903, 10)

# Modelo de Filtrado Colaborativo usando SVD

Utilizaremos SVD para hacer recomendaciones basadas en interacciones pasadas.


Funcionamiento:

SVD es una técnica matemática que descompone una matriz en tres matrices más pequeñas: U, Σ y V*. En el contexto de sistemas de recomendación, se usa para factorizar la matriz de usuario-ítem en componentes latentes, capturando patrones subyacentes en los datos.

Ventajas:

Es matemáticamente robusto y ha sido una técnica establecida durante mucho tiempo.

Puede capturar relaciones no evidentes en los datos.

Reducción de dimensionalidad: al capturar la esencia de los datos en factores latentes, se puede trabajar con dimensiones reducidas.

Desventajas:

No maneja bien datos faltantes. La matriz de usuario-ítem suele ser dispersa, y el SVD estándar no se diseñó para manejar matrices con muchos valores faltantes.

Puede ser computacionalmente costoso para matrices muy grandes.

Con esta modificación, primero se realiza una búsqueda de los mejores hiperparámetros para el algoritmo SVD utilizando GridSearchCV. Una vez encontrados los mejores hiperparámetros, el modelo se entrena y evalúa con estos.

In [ ]:
%%time
from surprise.model_selection import train_test_split

# Leer y procesar datos
reader = Reader()
data = Dataset.load_from_df(df[['reviewerID', 'asin', 'overall']], reader)

# Dividir el dataset
trainset, testset = train_test_split(data, test_size=0.3, random_state=10)

# Optimización de hiperparámetros usando GridSearchCV
param_grid = {
    'n_epochs': [5, 10, 20],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.02, 0.1, 0.5]
}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# Mejores hiperparámetros y sus correspondientes RMSE y MAE
print(gs.best_params['rmse'])
print(gs.best_score['rmse'])

# Entrenar el modelo SVD con los mejores hiperparámetros
svd = gs.best_estimator['rmse']
svd.fit(trainset)
predictions_svd = svd.test(testset)

# Función para calcular el Mean Absolute Percentage Error (MAPE)
def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Evaluar el modelo
rmse_svd = accuracy.rmse(predictions_svd, verbose=True)
mse_svd = mean_squared_error([pred.r_ui for pred in predictions_svd], [pred.est for pred in predictions_svd])
mae_svd = mean_absolute_error([pred.r_ui for pred in predictions_svd], [pred.est for pred in predictions_svd])
mape_svd = mape([pred.r_ui for pred in predictions_svd], [pred.est for pred in predictions_svd])

print(f"SVD - RMSE: {rmse_svd}, MSE: {mse_svd}, MAE: {mae_svd}, MAPE: {mape_svd}")

{'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.1}
1.080936999471357
RMSE: 1.0786
SVD - RMSE: 1.0785839307372322, MSE: 1.1633432956445788, MAE: 0.8076657455037248, MAPE: 33.50687027698853
CPU times: user 1h 21min 28s, sys: 1min 41s, total: 1h 23min 9s
Wall time: 1h 22min 16s


# Modelo de Filtrado Colaborativo usando Embedding con Keras:

Funcionamiento:

Los embeddings son representaciones vectoriales densas y de baja dimensión de ítems y/o usuarios. Estas representaciones capturan relaciones semánticas entre ítems o entre usuarios.

Un método popular para generar embeddings es la factorización de matrices, como la descomposición en valores singulares (SVD). En el contexto de sistemas de recomendación, se busca factorizar la matriz de interacciones usuario-ítem en dos matrices más pequeñas (una para los usuarios y otra para los ítems) cuyo producto aproximado reproduce la matriz original lo mejor posible.

Una vez que se han obtenido los embeddings, la predicción de una calificación o interacción entre un usuario e ítem se realiza tomando el producto escalar entre sus embeddings respectivos.

Ventajas:

Simplicidad y eficiencia en términos computacionales.

Puede manejar grandes conjuntos de datos debido a su naturaleza de baja dimensión.

Es efectivo para capturar patrones subyacentes en los datos.

Desventajas:

No tiene en cuenta características adicionales de usuarios o ítems.

Dificultades para manejar nuevos ítems o usuarios (problema de arranque en frío).

El código ahora incluye regularización en los embeddings, un callback para detener el entrenamiento si no hay mejoras (early stopping) y otro para reducir la tasa de aprendizaje si el error de validación no mejora (reduce learning rate on plateau).

In [7]:
data = df

In [8]:
# Crear un LabelEncoder para cada columna
reviewerID_encoder = LabelEncoder()
asin_encoder = LabelEncoder()

# Ajustar y transformar las columnas
data['reviewerID_encoded'] = reviewerID_encoder.fit_transform(data['reviewerID'])
data['asin_encoded'] = asin_encoder.fit_transform(data['asin'])

# Número de usuarios únicos y número de ítems únicos
n_users = len(np.unique(data['reviewerID_encoded']))
n_items = len(np.unique(data['asin_encoded']))

# Dimensiones del embedding
embedding_dim = 10

# Entradas
user_input = Input(shape=(1,))
item_input = Input(shape=(1,))

# Embeddings con regularización
user_embedding = Embedding(n_users, embedding_dim, embeddings_regularizer=l2(1e-6))(user_input)
item_embedding = Embedding(n_items, embedding_dim, embeddings_regularizer=l2(1e-6))(item_input)

# Producto punto para predecir la valoración/rating
merged = Dot(axes=2)([user_embedding, item_embedding])
merged = Flatten()(merged)

# Modelo
model = Model(inputs=[user_input, item_input], outputs=merged)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

verbose=0: No muestra ninguna barra de progreso ni métricas.

verbose=1: Muestra una barra de progreso y actualiza las métricas después de cada lote.

verbose=2: Muestra las métricas después de cada época, pero no muestra la barra de progreso.

In [9]:
from sklearn.model_selection import train_test_split

# Crear los arrays con los datos
user_ids = data['reviewerID_encoded'].values
item_ids = data['asin_encoded'].values
ratings = data['overall'].values

# División en entrenamiento y prueba
(user_ids_temp, user_ids_test, item_ids_temp, item_ids_test, ratings_temp, ratings_test) = train_test_split(
    user_ids, item_ids, ratings, test_size=0.2, random_state=42)

# División del conjunto de entrenamiento en entrenamiento y validación
(user_ids_train, user_ids_val, item_ids_train, item_ids_val, ratings_train, ratings_val) = train_test_split(
    user_ids_temp, item_ids_temp, ratings_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

In [10]:
# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=50)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=50, min_lr=1e-5)

In [11]:
%%time
# Entrenar el modelo con el conjunto de entrenamiento
model.fit([user_ids_train, item_ids_train], ratings_train, validation_data=([user_ids_val, item_ids_val], ratings_val), epochs=500, batch_size=4000, verbose=1, callbacks=[early_stop, reduce_lr])

Epoch 1/500
813/813 [==============================] - 12s 9ms/step - loss: 19.4894 - mae: 4.2571 - val_loss: 19.4436 - val_mae: 4.2515 - lr: 0.0010
Epoch 2/500
813/813 [==============================] - 7s 8ms/step - loss: 19.2143 - mae: 4.2200 - val_loss: 19.1354 - val_mae: 4.2036 - lr: 0.0010
Epoch 3/500
813/813 [==============================] - 7s 8ms/step - loss: 18.4571 - mae: 4.1079 - val_loss: 18.8070 - val_mae: 4.1373 - lr: 0.0010
Epoch 4/500
813/813 [==============================] - 7s 8ms/step - loss: 17.2804 - mae: 3.9304 - val_loss: 18.5069 - val_mae: 4.0701 - lr: 0.0010
Epoch 5/500
813/813 [==============================] - 7s 8ms/step - loss: 15.8485 - mae: 3.7047 - val_loss: 18.2263 - val_mae: 4.0000 - lr: 0.0010
Epoch 6/500
813/813 [==============================] - 7s 8ms/step - loss: 14.3455 - mae: 3.4501 - val_loss: 17.9616 - val_mae: 3.9268 - lr: 0.0010
Epoch 7/500
813/813 [==============================] - 7s 9ms/step - loss: 12.8924 - mae: 3.1840 - val_loss: 17

In [12]:
predictions = model.predict([user_ids_test, item_ids_test])

43518/43518 [==============================] - 55s 1ms/step


In [13]:
# Evaluar el modelo con el conjunto de prueba
loss, mae = model.evaluate([user_ids_test, item_ids_test], ratings_test, batch_size=40000)
print(f'Test Loss: {loss:.4f}')
print(f'Test MAE: {mae:.4f}')

35/35 [==============================] - 0s 3ms/step - loss: 8.7766 - mae: 1.8975
Test Loss: 8.7766
Test MAE: 1.8975


In [14]:
# MSE
mse = mean_squared_error(ratings_test, predictions)
print(f"MSE: {mse:.4f}")

# RMSE
rmse = np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

# MAE
mae = mean_absolute_error(ratings_test, predictions)
print(f"MAE: {mae:.4f}")

# MAPE (Mean Absolute Percentage Error)
mape = np.mean(np.abs((ratings_test - predictions.flatten()) / ratings_test)) * 100
print(f"MAPE: {mape:.2f}%")

MSE: 6.0313
RMSE: 2.4559
MAE: 1.8975
MAPE: 49.35%


# Modelo de Filtrado Colaborativo usando Red Neuronal Multicapa con Keras:

Funcionamiento:

Estos sistemas toman características de los ítems y/o usuarios y las pasan a través de una o varias capas de neuronas para obtener una predicción.
Las redes neuronales son capaces de capturar interacciones no lineales entre características, lo que las hace poderosas para tareas de modelado complejas.

Ventajas:

Capacidad de modelar relaciones no lineales.

Flexibilidad para incorporar múltiples fuentes de datos o características.

Puede manejar arranques en frío al incorporar características de nuevos ítems o usuarios.

Desventajas:

Mayor costo computacional en comparación con los sistemas basados únicamente en embeddings.

Riesgo de sobreajuste si no se tiene un conjunto de datos lo suficientemente grande.

Ahora el código incluye regularización en los embeddings, early stopping para detener el entrenamiento si el modelo deja de mejorar en el conjunto de validación, y reducción de la tasa de aprendizaje si el error en el conjunto de validación no mejora después de algunas épocas. Estas adiciones deberían ayudar a mejorar la capacidad de generalización del modelo.

In [15]:
df['user_id'] = df['reviewerID'].astype('category').cat.codes.values
df['item_id'] = df['asin'].astype('category').cat.codes.values

# Número de usuarios e ítems
n_users = df['user_id'].nunique()
n_items = df['item_id'].nunique()

# Hiperparámetros
hidden_units = [128, 64, 32]
dropout_rate = 0.2
l2_reg = 1e-6

# Arquitectura del modelo
user_input = Input(shape=[1], name='user_input')
item_input = Input(shape=[1], name='item_input')
concat = Concatenate()([user_input, item_input])
dense = concat
for units in hidden_units:
    dense = Dense(units, activation='relu', kernel_regularizer=l2(l2_reg))(dense)
    dense = Dropout(dropout_rate)(dense)
output = Dense(1, kernel_regularizer=l2(l2_reg))(dense)  # También puedes aplicar regularización L2 aquí
model = Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer=Adam(0.001), loss='mean_squared_error')

In [16]:
# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=50)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=50, min_lr=1e-5)

In [17]:
# Datos para entrenamiento
user_data = df['user_id'].values
item_data = df['item_id'].values
rating_data = df['overall'].values

# Dividir los datos
user_data_train, user_data_val, item_data_train, item_data_val, ratings_train, ratings_val = train_test_split(
    user_data, item_data, rating_data, test_size=0.3, random_state=42)

In [18]:
%%time
# Entrenamiento del modelo
history = model.fit([user_data_train, item_data_train], ratings_train,
    epochs=500,validation_data=([user_data_val, item_data_val], ratings_val),
    batch_size=4000,verbose=1,callbacks=[early_stop, reduce_lr])

Epoch 1/500
813/813 [==============================] - 6s 4ms/step - loss: 74484144.0000 - val_loss: 19.5621 - lr: 0.0010
Epoch 2/500
813/813 [==============================] - 3s 4ms/step - loss: 181593.7812 - val_loss: 19.5377 - lr: 0.0010
Epoch 3/500
813/813 [==============================] - 3s 4ms/step - loss: 86951.3281 - val_loss: 19.4924 - lr: 0.0010
Epoch 4/500
813/813 [==============================] - 3s 4ms/step - loss: 38759.3594 - val_loss: 19.4140 - lr: 0.0010
Epoch 5/500
813/813 [==============================] - 3s 4ms/step - loss: 22404.9082 - val_loss: 19.3016 - lr: 0.0010
Epoch 6/500
813/813 [==============================] - 3s 4ms/step - loss: 14300.4365 - val_loss: 19.1284 - lr: 0.0010
Epoch 7/500
813/813 [==============================] - 3s 4ms/step - loss: 12517.5596 - val_loss: 18.8727 - lr: 0.0010
Epoch 8/500
813/813 [==============================] - 3s 4ms/step - loss: 7769.8804 - val_loss: 18.4852 - lr: 0.0010
Epoch 9/500
813/813 [========================

In [19]:
# Evaluar el modelo
y_true = ratings_val
predictions = model.predict([user_data_val, item_data_val])

43518/43518 [==============================] - 64s 1ms/step


In [20]:
# MSE y RMSE
mse = mean_squared_error(y_true, predictions)
rmse = np.sqrt(mse)

# MAE
mae = mean_absolute_error(y_true, predictions)

# Funciones MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_idx = y_true != 0  # Para evitar divisiones por cero
    return np.mean(np.abs((y_true[non_zero_idx] - y_pred[non_zero_idx]) / y_true[non_zero_idx])) * 100

def compute_mape_by_batch(y_true, predictions, batch_size=50000):
    mape_sum = 0
    num_batches = int(np.ceil(len(y_true) / batch_size))

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size

        batch_y_true = y_true[start_idx:end_idx]
        batch_predictions = predictions[start_idx:end_idx]

        mape_sum += mean_absolute_percentage_error(batch_y_true, batch_predictions)

    return mape_sum / num_batches

# Luego llamas a la función
mape = compute_mape_by_batch(y_true, predictions)

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'MAPE: {mape}%')

MSE: 1.3605827693986772
RMSE: 1.1664402125264188
MAE: 0.9250949473115503
MAPE: 38.36821820901822%


# Modelo de Filtrado Colaborativo usando Red Neuronal Multicapa y Embedding con Keras:

Otra enfoque popular para filtrado colaborativo es utilizar redes neuronales multicapa, que esencialmente aprenden características no lineales de los datos. Vamos a construir un modelo que fusiona los embeddings de usuarios y artículos (por ejemplo, películas) en una red neuronal densa.

Este enfoque, que combina embeddings y redes neuronales multicapa, puede capturar interacciones más complejas y no lineales entre usuarios y artículos. Sin embargo, es crucial prestar atención al sobreajuste y asegurarse de que el modelo no esté simplemente memorizando los datos. Por lo tanto, es recomendable emplear técnicas de regularización, ajustar hiperparámetros y validar el rendimiento con un conjunto de datos de validación.

Funcionamiento:

Combina lo mejor de ambos mundos. Primero, se utilizan embeddings para convertir ítems y usuarios en representaciones vectoriales densas. Luego, estas representaciones se pasan a través de una red neuronal para hacer la predicción.

El proceso generalmente comienza con capas de embedding que convierten identificadores de usuarios e ítems en vectores. Estos vectores luego se pasan a través de capas densas para obtener la predicción final.

Ventajas:

Capacidad de capturar patrones subyacentes en los datos mediante embeddings y modelar interacciones no lineales mediante la red neuronal.

Flexibilidad para incorporar características adicionales.

Potencialmente más preciso que cualquiera de los otros dos métodos por separado.

Desventajas:

Mayor complejidad en el modelado y entrenamiento.

Mayor costo computacional.

Requiere un ajuste más cuidadoso y riesgo de sobreajuste si no se gestiona adecuadamente.

Regularización L2: Agregar una regularización L2 a las capas de embedding y densas para prevenir el sobreajuste.

Early Stopping: Añadir una callback de early stopping para detener el entrenamiento cuando no haya mejora en el conjunto de validación durante un cierto número de épocas.

Ajuste de la tasa de aprendizaje: Utilizar la callback ReduceLROnPlateau para reducir la tasa de aprendizaje cuando no haya mejora en el conjunto de validación.

Métricas adicionales: Puedes monitorizar otras métricas como el error absoluto medio (MAE) durante el entrenamiento.

In [21]:
# Convertir reviewerID y asin a índices numéricos secuenciales
df['user_id'] = df['reviewerID'].astype('category').cat.codes.values
df['item_id'] = df['asin'].astype('category').cat.codes.values

# Número de usuarios e ítems
n_users = df['user_id'].nunique()
n_items = df['item_id'].nunique()

# Hiperparámetros
n_latent_factors = 50
hidden_units = [128, 64]
dropout_rate = 0.2
l2_reg = 1e-4

# Arquitectura del modelo con regularización L2
user_embedding = Embedding(n_users, n_latent_factors, embeddings_regularizer=l2(l2_reg), name='user_embedding')(user_input)
item_embedding = Embedding(n_items, n_latent_factors, embeddings_regularizer=l2(l2_reg), name='item_embedding')(item_input)

user_vec = Flatten(name='flatten_users')(user_embedding)
item_vec = Flatten(name='flatten_items')(item_embedding)
concat = Concatenate()([user_vec, item_vec])
dense = concat
for units in hidden_units:
    dense = Dense(units, activation='relu', kernel_regularizer=l2(l2_reg))(dense)
    dense = Dropout(dropout_rate)(dense)
output = Dense(1, kernel_regularizer=l2(l2_reg))(dense)
model = Model(inputs=[user_input, item_input], outputs=output)

model.compile(optimizer=Adam(0.001), loss='mean_squared_error', metrics=['mae'])

In [22]:
# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=50, min_lr=1e-5)

In [23]:
# Entrenamiento
user_data = df['user_id'].values
item_data = df['item_id'].values
rating_data = df['overall'].values

# Usando validation_split, divide los datos
train_size = int(0.7 * len(user_data))
user_data_train, user_data_val = user_data[:train_size], user_data[train_size:]
item_data_train, item_data_val = item_data[:train_size], item_data[train_size:]
y_true = rating_data[train_size:]

In [24]:
%%time
history = model.fit([user_data_train, item_data_train], rating_data[:train_size],
                    epochs=500,validation_data=([user_data_val, item_data_val], y_true),
                    batch_size=4000,verbose=1,callbacks=[early_stop, reduce_lr])

Epoch 1/500
813/813 [==============================] - 29s 34ms/step - loss: 2.5592 - mae: 1.0725 - val_loss: 1.6737 - val_mae: 0.9378 - lr: 0.0010
Epoch 2/500
813/813 [==============================] - 27s 34ms/step - loss: 1.6569 - mae: 0.8723 - val_loss: 1.5840 - val_mae: 0.8885 - lr: 0.0010
Epoch 3/500
813/813 [==============================] - 27s 33ms/step - loss: 1.5034 - mae: 0.8137 - val_loss: 1.6316 - val_mae: 0.8801 - lr: 0.0010
Epoch 4/500
813/813 [==============================] - 27s 33ms/step - loss: 1.3262 - mae: 0.7610 - val_loss: 1.6692 - val_mae: 0.8760 - lr: 0.0010
Epoch 5/500
813/813 [==============================] - 27s 33ms/step - loss: 1.2176 - mae: 0.7166 - val_loss: 1.7002 - val_mae: 0.8936 - lr: 0.0010
Epoch 6/500
813/813 [==============================] - 27s 33ms/step - loss: 1.1684 - mae: 0.6940 - val_loss: 1.7327 - val_mae: 0.8759 - lr: 0.0010
Epoch 7/500
813/813 [==============================] - 27s 33ms/step - loss: 1.1431 - mae: 0.6806 - val_loss: 1.

In [25]:
# Predicciones
predictions = model.predict([user_data_val, item_data_val])

43518/43518 [==============================] - 64s 1ms/step


In [26]:
# MSE y RMSE
mse = mean_squared_error(y_true, predictions)
rmse = np.sqrt(mse)

# MAE
mae = mean_absolute_error(y_true, predictions)

# MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_idx = y_true != 0  # Para evitar divisiones por cero
    return np.mean(np.abs((y_true[non_zero_idx] - y_pred[non_zero_idx]) / y_true[non_zero_idx])) * 100

def compute_mape_by_batch(y_true, predictions, batch_size=50000):
    mape_sum = 0
    num_batches = int(np.ceil(len(y_true) / batch_size))

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size

        batch_y_true = y_true[start_idx:end_idx]
        batch_predictions = predictions[start_idx:end_idx]

        mape_sum += mean_absolute_percentage_error(batch_y_true, batch_predictions)

    return mape_sum / num_batches

# Luego llamas a la función
mape = compute_mape_by_batch(y_true, predictions)

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'MAPE: {mape}%')

MSE: 1.302656099763364
RMSE: 1.1413396075504276
MAE: 0.888510897368619
MAPE: 39.07414343406561%


Estas métricas te ayudarán a tener una mejor idea del desempeño de tu modelo. Por ejemplo:

MSE y RMSE son útiles cuando quieres penalizar grandes errores.

MAE te da una idea del error medio sin considerar la dirección del error.

MAPE es útil cuando quieres representar el error en términos porcentuales.

Para una evaluación completa, es recomendable utilizar un conjunto de validación aparte (es decir, no solo depender del validation_split). Esto asegura que estás evaluando el desempeño en datos que el modelo nunca ha visto durante el entrenamiento.

En resumen, la elección del tipo de sistema de recomendación dependerá de la naturaleza del conjunto de datos, las características disponibles, las capacidades computacionales y el tipo de relaciones o interacciones que se desean capturar. A menudo, un enfoque híbrido (como el sistema que combina embeddings y redes neuronales) ofrece un buen equilibrio entre precisión y eficiencia.